### Training the model

In [1]:
import IPython
from classes.vqa_model_trainer import VqaModelTrainer
from common.model_utils import get_trainable_params_distribution
from common.functions import get_highlighted_function_code
from common.settings import data_access
from common.utils import VerboseTimer
from data_access.model_folder import ModelFolder

Using TensorFlow backend.


In [2]:
import logging
import vqa_logger 
logger = logging.getLogger(__name__)

In [3]:
model_location = 'C:\\Users\\Public\\Documents\\Data\\2018\\vqa_models\\20190210_2236_21\\'
model_folder = ModelFolder(model_location)

### Loading the model to train:

In [4]:
epochs = 1
batch_size = 150
mt = VqaModelTrainer(model_folder, use_augmentation=True,batch_size=batch_size, data_access=data_access)

[2019-02-15 09:16:05][DEBUG] Loading Meta: 0:00:00.122232
[2019-02-15 09:16:05][DEBUG] Loading Meta: 0:00:00.122232
[2019-02-15 09:16:05][DEBUG] loading processed data from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-15 09:16:05][DEBUG] loading processed data from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-15 09:16:05][DEBUG] loading parquet from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-15 09:16:05][DEBUG] loading parquet from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-15 09:16:14][DEBUG] Loading parquet: 0:00:09.036892
[2019-02-15 09:16:14][DEBUG] Loading parquet: 0:00:09.036892
[2019-02-15 09:16:15][DEBUG] Converting to pandas: 0:00:01.475009
[2019-02-15 09:16:15][DEBUG] Converting to pandas: 0:00:01.475009


#### Lets take a look at the parameters:

In [5]:
get_trainable_params_distribution(mt.model)
# mt.model.summary()

Got a total of 191,815 trainable parameters


,index,layer,trainable_params,pretty_value
0,8,model_output_tanh_dense/kernel:0,135616,"135,616"
1,3,dense_1/kernel:0,36864,"36,864"
2,1,embbeding_LSTM/recurrent_kernel:0,16384,"16,384"
3,0,model_output_tanh_dense/bias:0,2119,"2,119"


#### And a look at data:

In [6]:
mt.df_meta_answers
mt.df_meta_words
mt.df_meta_answers.tail(2)

,answer
1706,pulmonary sequestration
1707,vestibular (acoustic) schwannoma


In [7]:
logger.debug(f"train Shape: {mt.data_train.shape}")
logger.debug(f"validation Shape: {mt.data_val.shape}")
mt.data_train.head(0)

[2019-02-15 09:16:16][DEBUG] train Shape: (12792, 12)
[2019-02-15 09:16:16][DEBUG] train Shape: (12792, 12)
[2019-02-15 09:16:16][DEBUG] validation Shape: (2000, 12)
[2019-02-15 09:16:16][DEBUG] validation Shape: (2000, 12)


,index,image_name,question,answer,path,processed_question,processed_answer,diagnosis,question_category,answer_embedding,question_embedding,group


### Overview of preperations for training:

##### The functions for getting the features & labels:

In [8]:
from common.functions import get_features, sentences_to_hot_vector, hot_vector_to_words
code_get_labels = get_highlighted_function_code(mt.get_labels, remove_comments=True)


code_get_features = get_highlighted_function_code(get_features, remove_comments=True)
code_hot_vector = get_highlighted_function_code(sentences_to_hot_vector, remove_comments=True)


print('Getting the label using a hot vector\n')
IPython.display.display(code_get_labels)
print('\n\nThe underlying method:\n')
IPython.display.display(code_hot_vector)


print('\n\nGetting the features using question embeding concatenation:\n')
IPython.display.display(code_get_features)

Getting the label using a hot vector





The underlying method:





Getting the features using question embeding concatenation:



##### Example of hot vector of answer (AKA answer...)

In [9]:
df = mt.data_train

class_df = mt.class_df
class_count = len(class_df)
# class_df.sample(5)

classes_indices_df = [class_df.loc[class_df.isin(ans.lower().split())] for ans in  df.answer]
classes_indices = [list(d.index) for d in classes_indices_df]

idx_sample = 9
print(df.answer[idx_sample])
classes_indices[idx_sample]

t2


[]

##### Will transform the sentences into vector and back using the following:

In [10]:
code = get_highlighted_function_code(hot_vector_to_words,remove_comments=False)
IPython.display.display(code)  

##### Check it looks sane by inversing the binarizing:

In [11]:
# words = mt.df_meta_words.word

arr_one_hot_vector = mt.get_labels(mt.data_train)
categorial_labels = arr_one_hot_vector

idx = 0
answer =  mt.data_train.answer.loc[idx]
print(f'The sentence:\n{answer}')

one_hot_vector = arr_one_hot_vector[idx]
label_words = hot_vector_to_words(one_hot_vector, mt.class_df)
print('\n\nThe reversed answer from hot vector:')
label_words

[2019-02-15 09:16:19][DEBUG] Using multi label
[2019-02-15 09:16:19][DEBUG] Using multi label
The sentence:
cta - ct angiography


The reversed answer from hot vector:


0                ct
20              cta
1537    angiography
Name: word, dtype: object

In [12]:
history = mt.train()

[2019-02-15 09:17:09][DEBUG] Getting train features: 0:00:48.721673
[2019-02-15 09:17:09][DEBUG] Getting train features: 0:00:48.721673
[2019-02-15 09:17:09][DEBUG] Using multi label
[2019-02-15 09:17:09][DEBUG] Using multi label
[2019-02-15 09:17:09][DEBUG] Getting validation labels: 0:00:00.213249
[2019-02-15 09:17:09][DEBUG] Getting validation labels: 0:00:00.213249
[2019-02-15 09:17:09][DEBUG] loading processed data from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-15 09:17:09][DEBUG] loading processed data from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-15 09:17:09][DEBUG] loading parquet from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-15 09:17:09][DEBUG] loading parquet from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-15 09:17:18][DEBUG] Loading parquet: 0:00:08.407950
[2019-02-15 0

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'


[2019-02-15 09:17:20][DEBUG] Getting 75 train features: 0:00:02.286615
[2019-02-15 09:17:20][DEBUG] Getting 75 train features: 0:00:02.286615
[2019-02-15 09:17:20][DEBUG] Using multi label
[2019-02-15 09:17:20][DEBUG] Using multi label
[2019-02-15 09:17:20][DEBUG] Getting 75 train labels: 0:00:00.013298
[2019-02-15 09:17:20][DEBUG] Getting 75 train labels: 0:00:00.013298
[2019-02-15 09:17:20][DEBUG] Expected shape: [(None, 12288, 1), (None, None, None, 3)]
[2019-02-15 09:17:20][DEBUG] Expected shape: [(None, 12288, 1), (None, None, None, 3)]
[2019-02-15 09:17:20][DEBUG] ---------------------------------------------------------------------------
[2019-02-15 09:17:20][DEBUG] ---------------------------------------------------------------------------
[2019-02-15 09:17:20][DEBUG] Actual training shape:((921600, 1), (75, 224, 224, 3))
[2019-02-15 09:17:20][DEBUG] Actual training shape:((921600, 1), (75, 224, 224, 3))
[2019-02-15 09:17:20][DEBUG] Train Labels shape:(75, 2119)
[2019-02-15 09:

ValueError: Error when checking input: expected embedding_input to have 3 dimensions, but got array with shape (24576000, 1)

### Save trained model:

In [ ]:
with VerboseTimer("Saving trained Model"):
    model_folder = mt.save(mt.model, mt.model_folder, history)


In [ ]:
print (model_folder.model_path)